In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../')
from ck_analysis import get_repo_id

repo_names = ["pay-as-you-go-api", "paygo-telepase-api", "prepaid-api", "sp-utilities-provider-api", "paygo-taggy-api"]

for repo_name in repo_names:
    df = pd.read_csv(f'../output/deploys/raw/{repo_name}_deploys.csv')
    df.columns = df.columns.str.strip()
    df["PERIOD_DATE"] = pd.to_datetime(df["period_date (Año y mes)"], format='%b %Y', errors='coerce', dayfirst=True)
    df["DEPLOYS"] = df["Deploys"]
    df["ROLLBACKED_DEPLOYS"] = df["Rollbacked Deploys"]
    df = df[['PERIOD_DATE', 'DEPLOYS', 'ROLLBACKED_DEPLOYS']]
    df.to_csv(f'../output/deploys/{get_repo_id(repo_name)}_deploys.csv', index=False)



In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.append('../')
from ck_analysis import get_repo_id

df = pd.read_csv('../output/static_analisys/raw/static_analisys_issues_master.csv')
df.columns = df.columns.str.strip()
df["CREATE_DTTM"] = pd.to_datetime(df["CREATE_DTTM"])
df["REPO_NAME"] = df["APPLICATION_NAME"].apply(get_repo_id)

severity_weights = {'low': 1, 'medium': 3, 'high': 5}
df['TOTAL_ISSUES_WEIGHTED'] = df['ISSUE_SEVERITY'].map(severity_weights) * df['TOTAL_ISSUES_QTY']

df = df.groupby(["CREATE_DTTM", "REPO_NAME", "PROVIDER"]).agg({"TOTAL_ISSUES_WEIGHTED": "sum", "TOTAL_ISSUES_QTY": "sum"}).reset_index()
df.to_csv('../output/static_analisys/static_analisys_master.csv', index=False)

In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../output/static_analisys/static_analisys_issues.csv')

# obtener los distintos valores de la combinacion de coliumnas PROVIDER y ISSUE_SEVERITY
df = df[["ISSUE_SEVERITY", "PROVIDER"]].drop_duplicates()
df.sort_values(by=["PROVIDER"], ascending=False)

In [6]:
import pandas as pd

def weighted_average(group, value_col, weight_col):
    return (group[value_col] * group[weight_col]).sum() / group[weight_col].sum()


df = pd.read_csv('../datasets/apps_ck_monthly.csv')
# Strip any leading or trailing spaces from column names
df.columns = df.columns.str.strip()

# Remove any rows where the type is enum
df = df[df['type'] != 'enum']
# Remove any rows where the class name contains ".dto." 
df = df[~df['class'].str.contains('.dto.')]

df = df[['tag_date', 'tag_name', 'class', 'wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'loc']]
df.columns = ['TagDate', 'TagName', 'ClassName', 'WMC', 'DIT', 'NOC', 'CBO', 'RFC', 'LCOM', 'LOC']
df['TagDate'] = pd.to_datetime(df['TagDate'])

grouped_metrics = df.groupby('TagDate')[['WMC', 'DIT', 'NOC', 'CBO', 'RFC', 'LCOM']]

median_by_date = grouped_metrics.median()
mean_by_date = grouped_metrics.mean()

# Crear un diccionario de funciones de agregación
agg_funcs = {metric: lambda x, col=metric: weighted_average(df.loc[x.index], col, 'LOC') 
            for metric in ['WMC', 'DIT', 'NOC', 'CBO', 'RFC', 'LCOM']}

weighted_avg_by_date = grouped_metrics.agg(agg_funcs)

weighted_avg_by_date


,WMC,DIT,NOC,CBO,RFC,LCOM
TagDate,,,,,,
2023-01-01,11.399131,1.123514,0.116796,12.295898,24.508946,37.818313
2023-02-01,12.030320,1.142123,0.124339,12.671759,26.518969,50.771846
2023-03-01,12.680000,1.141753,0.123740,12.682385,26.714619,51.833421
2023-04-01,11.668934,1.151258,0.119066,12.402693,24.822985,35.908700
2023-05-01,12.212518,1.151508,0.143883,12.762858,26.811237,52.540712
2023-06-01,12.908288,1.145839,0.129879,12.693781,26.990138,53.274650
2023-07-01,13.174679,1.180231,0.134893,12.911307,28.784464,61.652149
2023-08-01,12.261991,1.135133,0.123432,12.672377,26.818232,54.790773
2023-09-01,12.647439,1.148685,0.121426,12.472983,26.593099,54.107954
